In [5]:
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
import pandas as pd
import numpy as np 
import matplotlib as plt
import seaborn as sns

In [14]:
# read cv file
data = pd.read_csv('../stilligo_GitHub/Russian tourist_attractions.csv')
df = data

In [15]:
######## Transformation of latitude and longitude columns ##########
df = data
# slipe coodinate data (geolocalisation column) into "latitude" and "longitude" columns
df[['longitude', 'latitude']] = df['geolocation'].str.strip('()').str.split(', ', expand=True)

# Remove non-numeric characters and convert to numeric type
df['longitude'] = df['longitude'].str.replace(r'[^0-9. ]', '', regex=True).astype(str)
df['latitude']  = df['latitude'].str.replace(r'[^0-9. ]', '', regex=True).astype(str)

# change latitude and longitude type from str to float step
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')

# empty rows droped
df = df.dropna()
df = df.drop_duplicates()

#garder juste la colonne desiré
columns_to_drop = ['geolocation', 'name']  # List of columns to drop
df = df.drop(columns=columns_to_drop)

df.head()



,type,region,locality,longitude,latitude
0,architecture,Amur region,Blagoveshchensk,127.548877,49.811568
1,architecture,Sverdlovsk region,Ekaterinburg,60.621271,56.841798
2,architecture,Kursk region,Safonovka,35.139965,51.491293
3,architecture,Tomsk region,Tomsk,85.050946,56.469513
4,architecture,Novosibirsk region,Novosibirsk,82.958761,55.041787


In [47]:
####### pipeline  #######
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.neighbors import NearestNeighbors

# Impute then scale numerical values: 
num_transformer = Pipeline([
    ('labelencoder', LabelEncoder()), ('MinMaxScaler', MinMaxScaler())
])

# Encode categorical values
cat_transformer = OneHotEncoder(handle_unknown='ignore')

# Parallelize "num_transformer" and "cat_transfomer"
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['region','locality']),
    ('cat_transformer', cat_transformer, ['type'])
])

In [43]:
 # Visualizing Pipelines in HTML
from sklearn import set_config; set_config(display='diagram')
preprocessor

ColumnTransformer(transformers=[('num_transformer',
                                 Pipeline(steps=[('labelencoder',
                                                  LabelEncoder()),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler())]),
                                 ['region']),
                                ('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['type'])])

In [44]:
# Defining the features and the target

#example model
# X = data.drop(columns='charges')  
# y = data['charges']
X = df.drop(columns=['latitude','longitude'], axis = 1)
y = df[['latitude','longitude']]




In [45]:
# Train-Test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2723, 3), (681, 3), (2723, 2), (681, 2))

In [41]:
dtypeX_train

,type,region,locality
2357,"Monuments, sculptures, memorials",Altai region,Barnaul
3268,"Monuments, sculptures, memorials",Volgograd region,Volzhsky
2569,"Monuments, sculptures, memorials",Smolensk region,Smolensk
3978,"Monuments, sculptures, memorials",St. Petersburg,St. Petersburg
3936,"Monuments, sculptures, memorials",Tyva Republic,red
...,...,...,...
4536,monasteries,Lipetsk region,Roshinsky
3439,"Monuments, sculptures, memorials",Khanty-Mansi Autonomous Area,Nyagan
3507,"Monuments, sculptures, memorials",Krasnoyarsk region,Krasnoyarsk
349,architecture,Republic of Bashkortostan,Ufa


In [46]:
#fit the pipeline to the training data
X_train_transformed = preprocessor.fit_transform(X_train)

print("Original training set")
display(X_train.head(3))

print("Preprocessed training set")
display(df(X_train_transformed).head(3))

TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [ ]:
#score the knn model on the testing data
preprocessor.score(X_test,y_test)